In [1]:
import json
import pandas as pd
import numpy as np

import re

from sqlalchemy import create_engine
import psycopg2

# from config import db_password

import time

In [2]:
#  Add the clean movie function that takes in the argument, "movie".
def clean_movie(movie):
    movie = dict(movie) #create a non-destructive copy
    return movie

In [3]:
# 1 Add the function that takes in three arguments;
# Wikipedia data, Kaggle metadata, and MovieLens rating data (from Kaggle)

def movies(kaggle_metadata, MovieLens_rating,  wiki_movies_raw):
    # 2. Read in the kaggle metadata and MovieLens ratings CSV files as Pandas DataFrames.

    kaggle_metadata_df = pd.read_csv(kaggle_metadata, low_memory=False)

    MovieLens_rating_df = pd.read_csv(MovieLens_rating)

    
    with open(wiki_movies_raw, mode='r') as file:
        wiki_movies_raw = json.load(file)
    

    wiki_movies_df = pd.DataFrame(wiki_movies_raw)
    
    
    # Write a list comprehension to filter out TV shows.
    
    wiki_movies = [movie for movie in wiki_movies_raw
    
            if('Director' in movie or 'Directed by' in movie)
            and 'IMDb_link' in movie
            and 'No. of episodes' not in movie]
    

    # Write a list comprehension to iterate through the cleaned wiki movies list
    # and call the clean_movie function on each movie.
    
    clean_wiki_movies = [clean_movie(movies) for movies in wiki_movies]
    

    # Read in the cleaned movies list from Step 4 as a DataFrame.
                         
    wiki_movies_df = pd.DataFrame(clean_wiki_movies)


    # Write a try-except block to catch errors while extracting the IMDb ID using a regular expression string and
    #  dropping any imdb_id duplicates. If there is an error, capture and print the exception.
    
                         
    #try:    
    #except 


    #  Write a list comprehension to keep the columns that don't have null values from the wiki_movies_df DataFrame.
    wiki_movies_df['IMDb id'] = wiki_movies_df['IMDb_link'].str.extract(r'(tt\d{7})')
    wiki_movies_df.drop_duplicates(subset='IMDb id', inplace=True)

    # Create a variable that will hold the non-null values from the “Box office” column.
    
    nonnullbo = wiki_movies_df['Box office'].dropna()
    
    # Convert the box office data created in Step 8 to string values using the lambda and join functions.
    
    nonnullBO = nonnullBO.apply(lambda i: ' ' .join(i) if type(i) == list else i)
    

    # Write a regular expression to match the six elements of "form_one" of the box office data.
    
    form_one = r'\$\s*\d+\.?\d*\s*[mb]illi?on'
   
    # Write a regular expression to match the three elements of "form_two" of the box office data.
    
    form_two = r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)'

    # Add the parse_dollars function.
    
    def parse_dollars(s):
    # if s is not a string, return NaN
        if type(s) != str:
            return np.nan

    # if input is of the form $###.# million
        if re.match(r'\$\s*\d+\.?\d*\s*milli?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " million"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a million

        # return value

    # if input is of the form $###.# billion
        elif re.match(r'\$\s*\d+\.?\d*\s*billi?on', s, flags=re.IGNORECASE):

        # remove dollar sign and " billion"
            s = re.sub('\$|\s|[a-zA-Z]','', s)

        # convert to float and multiply by a billion

        # return value

    # if input is of the form $###,###,###
        elif re.match(r'\$\s*\d{1,3}(?:[,\.]\d{3})+(?!\s[mb]illion)', s, flags=re.IGNORECASE):

        # remove dollar sign and commas
            s = re.sub('\$|,','', s)

        # convert to float

        # return value

    # otherwise, return NaN
        else:
            return np.nan
    
    # Clean the box office column in the wiki_movies_df DataFrame.
    
    form_one_match = nonnullBO.str.contains(form_one, flags=re.IGNORECASE)
    form_two_match = nonnullBO.str.contains(form_two, flags=re.IGNORECASE)
    
    wiki_movies_df['nonnullBO'] = nonnullBO.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    wiki_movies_df.drop('Box office', axis=1, inplace=True)
    
    # Clean the budget column in the wiki_movies_df DataFrame.
    
    budget = ['Budget'].dropna()
    
    budget = budget.map(lambda x: ' ' .join(x) if type(x) == list else x)
    
    budget = budget.str.replace(r'\$.*[---](?![a-z])', '$', regex=True)
    
    form_one_match = budget.str.contains(form_one, flags=re.IGNORECASE)
    form_two_match = budget.str.contains(form_two, flags=re.IGNORECASE)
    
    budget[~form_one_match & ~form_two_match]
    
    budget = budget.str.replace(r'\[\d+\]\s*', '')
    budget[~form_one_match & ~form_two_match]
    
    wiki_movies_df['budget'] = budget.str.extract(f'({form_one}|{form_two})', flags=re.IGNORECASE)[0].apply(parse_dollars)
    
    
    # Clean the release date column in the wiki_movies_df DataFrame.
    
    release_date = wiki_movies_df['Release date'].dropna().apply(lambda i: ' ' .join(i) if type(i) == list else i)
    
    date_one = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s[123]\d,\s\d{4}'
    date_two = r'\d{4}.[01]\d.[123]\d'
    date_three = r'(?:January|February|March|April|May|June|July|August|September|October|November|December)\s\d{4}'
    date_four = r'\d{4}' 
    
    release_date.str.extract(f'({date_one}|{date_two}|{date_three}|{date_four})', flags=re.IGNORECASE)

    # Clean the running time column in the wiki_movies_df DataFrame.
    
    running_time = wiki_movies_df['Running time'].dropna().apply(lambda i: ' ' .join(i) if type(i) == list else i)
    
    running_time.str.contains(r'^\d*\s*m', flags=re.IGNORECASE).sum()
    
    running_time_extraction = running_time.str.extract(r'(\d+)\s*ho?u?r?s?\s*(\d*)|(\d+)\s*m')
    
    running_time_extraction = running_time_extraction.apply(lambda col: pd.to_numeric(col, errors='coerce')).fillna(0)
    
    wiki_movies_df['Running time'] = running_time_extraction.apply(lambda row: row[0]*60 + row[1] if row[2] == 0 else row[2], axis=1)
    
    wiki_movies_df.drop('Running time', axis=1, inplace=True)
     
    # 2. Clean the Kaggle metadata.
    kaggle_metadata[~kaggle_metadata['adult'].isin(['True','False'])]
    
    kaggle_metadata = kaggle_metadata[kaggle_metadata['adult']=='False'].drop('adult', axis='colums')
    
    kaggle_metadata['video'] = kaggle_metadata['video'] == 'True'
    
    kaggle_metadata['budget'] = kaggle_metadata['budget'].astype(int)
    kaggle_metadata['id'] = pd.to_numeric(kaggle_metadata['id'], errors='raise')
    kaggle_metadata['popularity'] = pd.to_numeric(kaggle_metadata['popularity'], errors='raise')
    kaggle_metadata['release_date'] = pd.to_datetime(kaggle_metadata['release_date'])
    ratings['timestamp'] = pd.to_datetime(ratings['timestamp'], unit='s')

    # 3. Merged the two DataFrames into the movies DataFrame.

    movies_dataframe = pd.merge(wiki_movies_df, kaggle_metadata, on='IMDb ID')

    # 4. Drop unnecessary columns from the merged DataFrame.
    
    movies_dataframe.drop(columns=['release_date_wiki', 'language', 'productioncompanies', 'Country', 'Story by'], inplace='True')


    # 5. Add in the function to fill in the missing Kaggle data.
    
    def missing_kaggle_data(df, kaggle_column, wiki_column):
        df[kaggle_column] = df.apply(lambda row: row[wiki_column] if row[kaggle_column] == 0 else row[kaggle_column], axis=1)
        df.drop(columns = wiki_column, inplace=True)
        

    # 6. Call the function in Step 5 with the DataFrame and columns as the arguments.
    
    missing_kaggle_data(movies_dataframe, 'runtime', 'running_time')
    missing_kaggle_data(movies_dataframe, 'budget_kaggle', 'budget_wiki')
    missing_kaggle_data(movies_dataframe, 'revenue', 'Box office')

    # 7. Filter the movies DataFrame for specific columns.


    # 8. Rename the columns in the movies DataFrame.


    # 9. Transform and merge the ratings DataFrame.
    
    return wiki_movies_df, movies_with_ratings_df, movies_df


SyntaxError: unexpected EOF while parsing (<ipython-input-3-a017aa26407d>, line 95)

In [4]:
# 10. Create the path to your file directory and variables for the three files.
file_dir = '../Resources'
# The Wikipedia data
wiki_file = f'{file_dir}/wikipedia.movies.json'
# The Kaggle metadata
kaggle_file = f'{file_dir}/movies_metadata.csv'
# The MovieLens rating data.
ratings_file = f'{file_dir}/ratings.csv'

In [5]:
# 11. Set the three variables equal to the function created in D1.
wiki_file, kaggle_file, ratings_file = extract_transform_load()

In [6]:
# 12. Set the DataFrames from the return statement equal to the file names in Step 11. 
wiki_movies_df = wiki_file
movies_with_ratings_df = kaggle_file
movies_df = ratings_file

In [7]:
# 13. Check the wiki_movies_df DataFrame. 
wiki_movies_df.head()

In [8]:
# 14. Check the movies_with_ratings_df DataFrame.
movies_with_ratings_df.head()

In [9]:
# 15. Check the movies_df DataFrame. 
movies_df.head()